In [12]:
import requests
import requests_cache
requests_cache.install_cache("bigdave") 
from lxml import html

In [5]:
s = requests.Session()

Want to start at the home page (maybe don't cache this).  
Find the first page named 'DT \d{5}'  

Go to that page.  
Crawl the clues.  
Find the nav links, follow them until the next DT page is found.  
Repeat until no more nav links?  
Be polite.  


In [8]:
base_url = "http://bigdave44.com"

In [10]:
r = s.get(base_url)
r.raise_for_status()

In [18]:
tree = html.fromstring(r.content)
links = tree.xpath('//h2[@class="entry-title"]/a[@rel="bookmark"]/@href')
print(links)

['http://bigdave44.com/2009/01/28/welcome/', 'http://bigdave44.com/2023/03/22/toughie-3030/', 'http://bigdave44.com/2023/03/22/dt-30254/', 'http://bigdave44.com/2023/03/22/sunday-toughie-59-review/', 'http://bigdave44.com/2023/03/22/st-3203-full-review/', 'http://bigdave44.com/2023/03/21/rip-pam-morton-mrs-bd/', 'http://bigdave44.com/2023/03/21/167421/', 'http://bigdave44.com/2023/03/21/dt-30253/', 'http://bigdave44.com/2023/03/20/dt-30252/', 'http://bigdave44.com/2023/03/20/rookie-corner-467/', 'http://bigdave44.com/2023/03/19/sunday-toughie-60-hints/']
